In [5]:
from pathlib import Path
import multiprocessing

import pandas as pd
import numpy as np

import IMP
import IMP.atom

import sys
sys.path.append(str(Path(Path.home(), "Documents/xray/src")))
# from cctbx_score import get_score
from score import pool_score
from miller_ops import get_miller_array, clean_miller_array

In [13]:
pdb_file = Path("/Users/matthew/Documents/xray/dev/45_synthetic_native_4/data/pdbs/native.pdb")
# w_mat = np.array([[0.9, 0.1], [0.1, 0.9]])
w_mat = np.array([[1, 0], [0, 1]])

# ref_w_mat = np.array([[0.9, 0.1], [0.1, 0.9]])
ref_w_mat = np.array([[0.9, 0.9], [0.1, 0.1]])

# cif_files = [Path("/Users/matthew/Documents/xray/dev/45_synthetic_native_4/data/cifs/native_0.cif"), Path("/Users/matthew/Documents/xray/dev/45_synthetic_native_4/data/cifs/native_1.cif")]
cif_files = [Path("/Users/matthew/Documents/xray/dev/45_synthetic_native_4/data/cifs/native_0.cif"), Path("/Users/matthew/Documents/xray/dev/45_synthetic_native_4/data/cifs/native_0.cif")]

# cond_names = ["native_0", "native_1"]
cond_names = ["state_0", "state_1"]


In [14]:
natives_df = pd.DataFrame()

natives_df.loc[0, "pdb"] = str(pdb_file)
natives_df.loc[0, "cif_0"] = str(cif_files[0])
natives_df.loc[0, "cif_1"] = str(cif_files[1])

for i in range(2):
    for j in range(2):
        natives_df.loc[0, "w_{}_{}".format(i, j)] = w_mat[i, j]

In [15]:
param_dict = dict()

param_dict["decoy_file"] = pdb_file
param_dict["decoy_w_mat"] = w_mat
param_dict["ref_file"] = pdb_file
param_dict["ref_w_mat"] = ref_w_mat
param_dict["score_fs"] = ["xray_0", "rmsd_0", "xray_1", "rmsd_1", "ff"]

param_dict["cif_files"] = cif_files
param_dict["res"] = 0
param_dict["scale"] = True
param_dict["scale_k1"] = True

score_dict = pool_score(param_dict)

print(score_dict)



SETTING UP MULTI STATE MULTI CONDITION MODEL
/Users/matthew/Documents/xray/dev/45_synthetic_native_4/data/pdbs/native.pdb
NO STATES:  2
NO CONDITIONS:  2
NO OF ATOMS PER STATE:  2464
SETTING UP MULTI STATE MULTI CONDITION MODEL
/Users/matthew/Documents/xray/dev/45_synthetic_native_4/data/pdbs/native.pdb
NO STATES:  2
NO CONDITIONS:  2
NO OF ATOMS PER STATE:  2464
bnd 8407.17710428151
ang 459.9314170772075
dih 846.704758967607
imp 35.78119311958154
nbd -168.06408877078033
bnd 8466.039890660384
ang 444.9792810070139
dih 840.1603736606831
imp 39.59712928429329
nbd -760.1477642534699
{'native': 1, 'pdb_file': '/Users/matthew/Documents/xray/dev/45_synthetic_native_4/data/pdbs/native.pdb', 'r_free_0': 0.07049606778261769, 'r_work_0': 0.07107143095247746, 'cif_files': [PosixPath('/Users/matthew/Documents/xray/dev/45_synthetic_native_4/data/cifs/native_0.cif'), PosixPath('/Users/matthew/Documents/xray/dev/45_synthetic_native_4/data/cifs/native_0.cif')], 'xray_0': 2.959052001975915, 'rmsd_0': 0

In [16]:

for cond in range(2):
    cond_name = cond_names[cond]

    natives_df.loc[0, "xray_{}".format(cond_name)] = score_dict["xray_{}".format(cond)]
    natives_df.loc[0, "r_free_{}".format(cond_name)] = score_dict["r_free_{}".format(cond)]
    natives_df.loc[0, "r_work_{}".format(cond_name)] = score_dict["r_work_{}".format(cond)]
    natives_df.loc[0, "rmsd_{}".format(cond_name)] = score_dict["rmsd_{}".format(cond)]
    natives_df.loc[0, "ff".format(cond_name)] = score_dict["ff"]

In [17]:
natives_df

,pdb,cif_0,cif_1,w_0_0,w_0_1,w_1_0,w_1_1,xray_state_0,r_free_state_0,r_work_state_0,rmsd_state_0,ff,xray_state_1,r_free_state_1,r_work_state_1,rmsd_state_1
0,/Users/matthew/Documents/xray/dev/45_synthetic...,/Users/matthew/Documents/xray/dev/45_synthetic...,/Users/matthew/Documents/xray/dev/45_synthetic...,1.0,0.0,0.0,1.0,2.959052,0.070496,0.071071,0.028156,18612.159295,4.794645,0.294948,0.308332,0.2534


In [18]:
natives_df.to_csv("../data/csvs/natives_1_state.csv")